In [1]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import pickle
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
import pickle

import warnings

# Ignora gli avvisi di InconsistentVersionWarning
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# Il resto del tuo codice continua da qui...

####################################################### Metodi da implementare #######################################################

def getName():
    return "Andrea Accornero (1097521), Alessandro Tocco (1097880)"


def preprocess(df, clfName):
    
    if((clfName == 'LR') or (clfName == 'FF')):
        # Features
        X = df.drop("Year", axis=1)
        y = df["Year"]
        scaler = pickle.load(file=open("standardScaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([X, y], axis = 1)
        return dfNew  
    
    if((clfName == 'RF') or (clfName == 'KNR')):
        X = df.drop("Year", axis=1)
        y = df["Year"]
        scaler = pickle.load(open("minMaxScaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([X, y], axis = 1)
        return dfNew
    
    if((clfName == 'TB')or (clfName == 'TF')):
        X = df.drop("Year", axis=1)
        y = df["Year"]
        dfNew = pd.concat([X, y], axis = 1)
        return dfNew    
    
    
            
def load(clfName):
    if (clfName == "LR"):
        clf = pickle.load(open("regression.save", 'rb'))
        return clf
    elif (clfName == "RF"):
        clf = pickle.load(open("randomForestReg.save", 'rb'))
        return clf
    elif (clfName == "KNR"):
        clf = pickle.load(open("kNeighborReg.save", 'rb'))
        return clf
    elif (clfName == "SVR"):
        clf = pickle.load(open("supportVectorReg.save", 'rb'))
        return clf
    elif (clfName == "FF"):
        clf = pickle.load(open("feedForwardNN.save", 'rb'))
        return clf
    elif (clfName == "TB"):
        clf = torch.load(open("tabNet.save", 'rb'))
        return clf
    elif (clfName == "TF"):
        clf = torch.load(open("tabTransf.save", 'rb'))
        return clf
    else:
        return None
    
    

def predict(df, clfName, clf):
    
    if(clfName == 'FF'):
        X = df.drop("Year", axis=1).values
        y = df["Year"].values
        X_tensor = torch.tensor(X, dtype=torch.float32)
        clf.eval()
        with torch.no_grad():
            y_pred = clf(X_tensor).squeeze().numpy()
        
        mse = mean_squared_error(y_true=y, y_pred=y_pred)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        mape = mean_absolute_percentage_error(y_true=y, y_pred=y_pred)
        r2 = r2_score(y, y_pred)
        perf = {"mse": mse, "mae": mae, "mape": mape, "r2square": r2}
        return perf
    
    if((clfName == 'LR') or (clfName == 'KNR')):
        X = df.drop("Year", axis=1)
        y = df["Year"]
        ypred = clf.predict(X)
        mse = mean_squared_error(y_true=y, y_pred=ypred)
        mae = mean_absolute_error(y_true=y, y_pred=ypred)
        mape = mean_absolute_percentage_error(y_true=y, y_pred=ypred)
        r2 = r2_score(y_true=y, y_pred=ypred)
        perf = {"mse": mse, "mae": mae, "mape": mape, "r2square": r2}
        return perf
    
    if((clfName == 'TB' ) or (clfName == 'TF')):
        X = df.drop("Year", axis=1)
        y = df["Year"]
        y_pred = clf.predict(X)  # Utilizza il modello TabNet per fare le previsioni
        mse = mean_squared_error(y_true=y, y_pred=y_pred)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        mape = mean_absolute_percentage_error(y_true=y, y_pred=y_pred)
        r2 = r2_score(y_true=y, y_pred=y_pred)
        perf = {"mse": mse, "mae": mae, "mape": mape, "r2square": r2}
        return perf

        

#####################################################################################################################################

FILENAME = "train.csv"
CLF_NAME_LIST = [
                 "TF",
                 "TB",
                 "LR", 
                 "KNR",
                 "FF",
                 ]
df = pd.read_csv(FILENAME)

#Esecuzione degli algoritmi
for modelName in CLF_NAME_LIST:
    dfProcessed = preprocess(df, modelName)
    clf = load(modelName)
    perf = predict(dfProcessed, modelName, clf)
    print("RESULT team: "+str(getName())+" algoName: "+ modelName + " perf: "+ str(perf))



RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: TF perf: {'mse': 3769354.610894032, 'mae': 1941.3243129112032, 'mape': 0.9714606717896227, 'r2square': -34203.02053287905}
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: TB perf: {'mse': 3746392.993148875, 'mae': 1935.4529982801512, 'mape': 0.9685264973087812, 'r2square': -33994.66135049976}
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: LR perf: {'mse': 84.44611870757275, 'mae': 6.627242329639595, 'mape': 0.0033265999583101764, 'r2square': 0.23371582767834176}
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: KNR perf: {'mse': 71.39989981376897, 'mae': 6.0878026622926, 'mape': 0.0030545753454395692, 'r2square': 0.3521003218382742}
RESULT team: Andrea Accornero (1097521), Alessandro Tocco (1097880) algoName: FF perf: {'mse': 87.53516585243662, 'mae': 6.739812542985402, 'mape': 0.003381688696735196, 'r2square': 0.20568507